## 3.2.2 Realistic simulation setup
In this script we will change the paradigm. Instead of only using 4 points in the real world to define a circle in the 2D plane, we will use multiple points(~1024).

### import  libraries

In [1]:
vscode = 1

In [2]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
 
from typing import Sequence
from calib_lib import *
from matplotlib.collections import PatchCollection      # pathcoletion for multiple patches
import matplotlib.cm as cm                              # for colormaps
from matplotlib.patches import Rectangle                # for rectangle representation
from matplotlib.patches import Ellipse                  # for ellipse representation   
from pixelization_lib import *      

DECIMALS = 2            # how many decimal places to use in print


In [4]:
F = 16                                       # focal length( in mm )
gridSize = 1024
subpixel_grid = 64
image_size = np.array([1936,1216])               # sensor size(in mm)
#image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                                 # yaw angle

# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44

radius = 5

## play with the

In [5]:
xrange_5 = np.linspace(-6000,6000,1)
yrange_5 = np.linspace(-4000,4000,1)
zrange_5 = ([20000])


world_pts_arr,x_zhang_R,x_zhang_L,P_R,P_L = create_1024_point_chess(xrange_5,yrange_5,zrange_5)

(1, 140, 1024, 3)
(140, 1024, 4)
(1, 140, 1024, 3)
World points:  (1, 1, 143360, 3)
Image points:  (1, 140, 1024, 2)


In [6]:
def spot_center_coord(cx0,cy0,pixelGridX,pixelGridY):
    cx0_max = np.where((cx0 < pixelGridX))
    cx0_min = np.where((cx0 > pixelGridX))
    #print(cx0_max[0][0])
    #print(cx0_min[0][-1])
    cy0_max = np.where((cy0 < pixelGridY))
    cy0_min = np.where((cy0 > pixelGridY))
    #print(cy0_max[0][0])
    #print(cy0_min[0][-1])

    return cx0_max[0][0],cx0_min[0][-1],cy0_max[0][0],cy0_min[0][-1]

In [7]:
def pixel_cover_finder(gridSize,cx0max,cy0max,cx0,cy0,r):
    # array for percentage storage
    pixel_covered_percent = np.zeros((1,))

    xgrid = np.linspace(0,1,gridSize)*pixel_width
    ygrid = np.linspace(0,1,gridSize)*pixel_width

    x_pixel = cx0max+xgrid
    y_pixel = cy0max+ygrid

    xx_pixel, yy_pixel = np.meshgrid(x_pixel,y_pixel)

    circ_total = np.sqrt((xx_pixel-cx0)**2 + (yy_pixel-cy0)**2) <= r
    circ = np.where(circ_total == True)

    pixel_covered = circ[0].shape
    totalpixel_area = gridSize**2
    pixel_covered_percent = (pixel_covered[0]/totalpixel_area)*100
    #print("pixel cover percentage: " + str(pixel_covered_percent) + " %")

    return pixel_covered_percent

In [8]:
x0_L = x_zhang_L[:,:,:,0]*pixel_width
y0_L = x_zhang_L[:,:,:,1]*pixel_width

In [9]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) * pixel_width
pixelGridY  = (np.arange(0,image_size[1])) * pixel_width
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

points_max_min_L =np.zeros((x0_L.shape[0],x0_L.shape[1],x0_L.shape[2],2))

for i in range(x0_L.shape[0]):
    for j in range(x0_L.shape[1]):
        for k in range(x0_L.shape[2]):

            cx0max,cx0min,cy0max,cy0min = spot_center_coord(x0_L[i,j,k],y0_L[i,j,k],pixelGridX,pixelGridY)

            # max and min points in (x,y) coordinates
            cx0min_pxl_L = (cx0min*pixel_width)
            cx0max_pxl_L = (cx0max*pixel_width)
            cy0min_pxl_L = (cy0min*pixel_width)
            cy0max_pxl_L = (cy0max*pixel_width)

            #array max and min points
            points_max_min_L[i,j,k,:] = (([cx0min_pxl_L,cy0min_pxl_L]))


In [10]:
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(pixelGridXX,pixelGridYY,'r')
ax.plot(pixelGridXX.T,pixelGridYY.T,'b')
for chess_b_i in range(1):
    for point in range(1):
        ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
        max_pt_x = np.max(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        max_pt_y = np.max(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        min_pt_x = np.min(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        min_pt_y = np.min(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        #ax.plot(np.array(points_outter_L[chess_b_i][point])[:,0],np.array(points_outter_L[chess_b_i][point])[:,1], color = 'y',ls = "None", marker = "*")
        ax.plot(max_pt_x,max_pt_y, color = 'r',ls = "None", marker = "*")
        ax.plot(max_pt_x,min_pt_y, color = 'r',ls = "None", marker = "*")
        ax.plot(min_pt_x,min_pt_y, color = 'r',ls = "None", marker = "*")
        ax.plot(min_pt_x,max_pt_y, color = 'r',ls = "None", marker = "*")

        pts = np.array([[max_pt_x,max_pt_y],[max_pt_x,min_pt_y],[min_pt_x,min_pt_y],[min_pt_x,max_pt_y]])
        cmin_pxl_L = np.zeros((pts.shape[0],pts.shape[1]))
        for pts_i,pts_d in enumerate(pts):
            cx0max,cx0min,cy0max,cy0min  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

            # max and min points in (x,y) coordinates
            cmin_pxl_L[pts_i,0] = (cx0min*pixel_width)
            cmin_pxl_L[pts_i,1] = (cy0min*pixel_width)

        x_L_pts = np.arange(cmin_pxl_L[3,0],cmin_pxl_L[0,0]+pixel_width,pixel_width)
        y_L_pts = np.arange(cmin_pxl_L[1,1],cmin_pxl_L[0,1]+pixel_width,pixel_width)  

        #ax.set_xlim(x_L_pts[-1],x_L_pts[-1]+pixel_width)
        #ax.set_ylim(y_L_pts[-2],y_L_pts[-2]+pixel_width)

        
        #ax.set_xlim(max(x0_L[0,point,:]),min(x0_L[0,point,:]))
        #ax.set_ylim(max(y0_L[0,point,:]),min(y0_L[0,point,:]))
        #ax.set_aspect(1)
plt.show()

## Define the outter mesh points
In this part, we use the x_L and y_L and composed them into a mesh grid that respects the grid

In [11]:
pts = np.array([[max_pt_x,max_pt_y],[max_pt_x,min_pt_y],[min_pt_x,min_pt_y],[min_pt_x,max_pt_y]])
cmin_pxl_L = np.zeros((pts.shape[0],pts.shape[1]))
for pts_i,pts_d in enumerate(pts):
    cx0max,cx0min,cy0max,cy0min  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

    # max and min points in (x,y) coordinates
    cmin_pxl_L[pts_i,0] = (cx0min*pixel_width)
    cmin_pxl_L[pts_i,1] = (cy0min*pixel_width)

x_L_pts = np.arange(cmin_pxl_L[3,0],cmin_pxl_L[0,0]+pixel_width,pixel_width)
y_L_pts = np.arange(cmin_pxl_L[1,1],cmin_pxl_L[0,1]+pixel_width,pixel_width)


#define the corners of the spot
[X_crns,Y_crns] = np.meshgrid(x_L_pts,y_L_pts)

In [12]:
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(pixelGridXX,pixelGridYY,'r')
ax.plot(pixelGridXX.T,pixelGridYY.T,'b')
for chess_b_i in range(1):
    for point in range(1):
        ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
        max_pt_x = np.max(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        max_pt_y = np.max(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        min_pt_x = np.min(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        min_pt_y = np.min(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        ax.plot(X_crns,Y_crns, color = 'r',ls = "None", marker = "*")

plt.show()

In [25]:
[X_crns,Y_crns] = np.meshgrid(x_L_pts,y_L_pts)

In [26]:
X_crns.shape

(3, 1)

In [28]:
from matplotlib.path import Path

xgrid = np.linspace(0,1,gridSize)*pixel_width
ygrid = np.linspace(0,1,gridSize)*pixel_width

lst = []
ptchs = []
for i in range(X_crns.flatten().shape[0]):

    cx0max = X_crns.flatten()[i]
    cy0max = Y_crns.flatten()[i]


    x_pixel = cx0max+xgrid
    y_pixel = cy0max+ygrid
    xx_pixel, yy_pixel = np.meshgrid(x_pixel,y_pixel)

    x, y = xx_pixel.flatten(), yy_pixel.flatten()
    points = np.vstack((x,y)).T 

    x_L = x_zhang_L[0,0,:,:]*pixel_width
    x_L = tuple(x_L)
    p = Path(x_L) # make a polygon
    grid = p.contains_points(points)

    xxgrid = np.where(grid == True)

    lst.append((xxgrid[0].shape[0]/(gridSize**2))*100)


    ptchs.append(Rectangle((cx0max,cy0max), pixel_width,pixel_width))



In [29]:
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(pixelGridXX,pixelGridYY,'r')
ax.plot(pixelGridXX.T,pixelGridYY.T,'b')
for chess_b_i in range(1):
    for point in range(1):
        ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
        max_pt_x = np.max(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        max_pt_y = np.max(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        min_pt_x = np.min(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        min_pt_y = np.min(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        ax.plot(X_crns,Y_crns, color = 'r',ls = "None", marker = "*")

        ####### original centroid ######
        centroid_L_x_orig = np.sum(x_zhang_L[chess_b_i,point,:,0]*pixel_width)/x_zhang_L[chess_b_i,point,:,0].shape[0]
        centroid_L_y_orig = np.sum(x_zhang_L[chess_b_i,point,:,1]*pixel_width)/x_zhang_L[chess_b_i,point,:,0].shape[0]

        ax.plot(centroid_L_x_orig,centroid_L_y_orig,'g*')
p = PatchCollection(ptchs, cmap=cm.jet, alpha=1)
pixel_covered_percent_arr = (np.array(lst)).flatten()


####### estimated centroid #####
centroid_L_x = np.sum(np.multiply(X_crns.flatten(),lst))/(np.sum(lst)) + pixel_width/2
centroid_L_y = np.sum(np.multiply(Y_crns.flatten(),lst))/(np.sum(lst)) + pixel_width/2
ax.plot(centroid_L_x,centroid_L_y,'y.')





# define the color
p.set_array(pixel_covered_percent_arr)
# set color max and min
p.set_clim([0, 100])
ax.add_collection(p)

fig.colorbar(p)
        

plt.show()